In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
import numpy as np
from evaluate import load

In [2]:
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/andrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
metric = load("rouge")

In [4]:
max_input = 1024
max_target = 128
# model_checkpoint = "./bart-large-cnn-finetuned/checkpoint-10650/"
model_checkpoint = "../BART-SFT/checkpoint-150/"

In [5]:
raw_datasets = load_from_disk("../data/hf_dataset")

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [8]:
def preprocess_data(examples):
    # get all the articles, prepend each with "bias;"
    inputs = [
        f"{bias}; {article}"
        for bias, article in zip(examples["summary_bias"], examples["article"])
    ]
    # tokenize the inputs
    model_inputs = tokenizer(
        inputs, max_length=max_input, padding="max_length", truncation=True
    )

    # tokenize the summaries
    targets = tokenizer(
        examples["summary"],
        max_length=max_target,
        padding="max_length",
        truncation=True,
    )

    # set labels
    model_inputs["labels"] = targets["input_ids"]
    # return the tokenized data
    # input_ids, attention_mask and labels
    return model_inputs


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        use_aggregator=True,
    )
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [9]:
tokenized_data = raw_datasets.map(preprocess_data, batched=True)

Map:   0%|          | 0/4664 [00:00<?, ? examples/s]

Map:   0%|          | 0/542 [00:00<?, ? examples/s]

Map:   0%|          | 0/602 [00:00<?, ? examples/s]

In [33]:
batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"BART-SFT",
    evaluation_strategy="steps",
    eval_steps=150,
    warmup_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    log_level="info",
    logging_dir="./log",
    logging_first_step=True,
    logging_steps=5,
    save_total_limit=3,
    save_strategy="steps",
    save_steps=150,
    load_best_model_at_end=True,
    num_train_epochs=25,
    predict_with_generate=True,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices


ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA or NPU devices or certain XPU devices (with IPEX).

In [12]:
from collections import defaultdict
matches = defaultdict(list)
for item in tokenized_data["test"]:
    matches[f"{item['id']}_{item['article_bias']}"].append(item)

In [13]:
keys = []
for key, val in matches.items():
    if len(val) == 3:
        keys.append(key)

In [14]:
keys

['5010_center',
 '5010_left',
 '5010_right',
 '5037_center',
 '5037_left',
 '5037_right',
 '612_center',
 '612_left',
 '612_right',
 '4972_center',
 '4972_left',
 '4972_right',
 '4661_center',
 '4661_left',
 '4661_right',
 '4892_center',
 '4892_left',
 '4892_right',
 '1866_center',
 '1866_left',
 '1866_right',
 '5012_center',
 '5012_left',
 '5012_right',
 '4465_center',
 '4465_left',
 '4465_right',
 '4186_center',
 '4186_left',
 '4186_right',
 '3678_center',
 '3678_left',
 '3678_right',
 '3242_center',
 '3242_left',
 '3242_right',
 '3234_center',
 '3234_left',
 '3234_right',
 '3711_center',
 '3711_left',
 '3711_right',
 '4845_center',
 '4845_left',
 '4845_right',
 '4217_center',
 '4217_left',
 '4217_right',
 '319_center',
 '319_left',
 '319_right',
 '5020_center',
 '5020_left',
 '5020_right',
 '4162_center',
 '4162_left',
 '4162_right',
 '3927_center',
 '3927_left',
 '3927_right',
 '2231_center',
 '2231_left',
 '2231_right',
 '4305_center',
 '4305_left',
 '4305_right',
 '5219_center',


In [15]:
len(tokenized_data["test"])

602

In [30]:
[t['article_bias'] for t in test]

['center', 'left', 'right']

In [17]:
test = tokenized_data["test"].select([0,1,2])

In [26]:
preds2 = trainer.predict(test)

The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: article, summary_bias, summary, id, article_bias. If article, summary_bias, summary, id, article_bias are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3
  Batch size = 4
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [27]:
preds2.metrics

{'test_loss': 2.01328444480896,
 'test_rouge1': 26.0676,
 'test_rouge2': 5.2797,
 'test_rougeL': 13.9776,
 'test_rougeLsum': 19.1916,
 'test_gen_len': 97.0,
 'test_runtime': 58.9905,
 'test_samples_per_second': 0.051,
 'test_steps_per_second': 0.017}

In [32]:
predicted_texts = [tokenizer.decode(token_ids, skip_special_tokens=True) for token_ids in preds2.predictions]

# Show a few example outputs
for i in range(len(predicted_texts)):  # Just show the first 5 examples
    print(f"Example {i+1}:")
    print(f"Article Bias: {test[i]['article_bias']}, Summary Bias: {test[i]['summary_bias']}")
    print(f"Input: {test[i]['article']}")
    print()
    print(f"Target Summary: {test[i]['summary']}")
    print()
    print(f"Predicted Summary: {predicted_texts[i]}\n")
    print()
    print('*' * 100)

Example 1:
Article Bias: center, Summary Bias: left
Input: U.S. House condemns 'QAnon' conspiracy theory; 17 Republicans vote no
WASHINGTON (Reuters) - The U.S. House of Representatives on Friday voted to condemn the online pro-Trump conspiracy theory known as "QAnon," but 17 Republicans opposed the non-binding resolution, whose sponsor Democrat Representative Tom Malinowski said he has received death threats.
The House voted 371-18 to reject the conspiracy theory, which posits President Donald Trump has been working to take down a global child sex ring. As many as a dozen Republican candidates for Congress have voiced some support for the theory, and at least one of them appears to be a on a path to victory.
"The grotesque nature of the tweets and Instagram posts and the anti-Semitic tripe spewed by QAnon adherents should cause concern for everyone," Representative Denver Riggleman, a Republican co-sponsor of the resolution, said on the House floor.
"But the death threats Tom Malinows

In [45]:
preds = trainer.predict(tokenized_data["test"])

The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, summary_bias, article, article_bias. If summary, id, summary_bias, article, article_bias are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 581
  Batch size = 4


In [49]:
preds.metrics

{'test_loss': 0.22916316986083984,
 'test_rouge1': 90.1272,
 'test_rouge2': 85.6914,
 'test_rougeL': 87.6164,
 'test_rougeLsum': 89.4535,
 'test_gen_len': 123.9088,
 'test_runtime': 764.4302,
 'test_samples_per_second': 0.76,
 'test_steps_per_second': 0.191}